In [1]:
import requests

In [2]:
from haystack.nodes import PDFToTextConverter,PreProcessor
from haystack.utils import convert_files_to_docs,clean_wiki_text

In [3]:
url = "search-auto-insurance-nbdvjfxbsyudd5yhpq3e4ic6bu.ap-south-1.es.amazonaws.com"
username = "tarunm"
password = "May#2022password"

In [4]:
requests.delete("https://search-auto-insurance-nbdvjfxbsyudd5yhpq3e4ic6bu.ap-south-1.es.amazonaws.com/auto-insurance/",auth=(username, password)).json()

{'acknowledged': True}

In [5]:
requests.get('https://search-auto-insurance-nbdvjfxbsyudd5yhpq3e4ic6bu.ap-south-1.es.amazonaws.com/_cluster/health',auth=(username, password)).json()

{'cluster_name': '201328346379:auto-insurance',
 'status': 'green',
 'timed_out': False,
 'number_of_nodes': 3,
 'number_of_data_nodes': 3,
 'discovered_master': True,
 'active_primary_shards': 8,
 'active_shards': 17,
 'relocating_shards': 0,
 'initializing_shards': 0,
 'unassigned_shards': 0,
 'delayed_unassigned_shards': 0,
 'number_of_pending_tasks': 0,
 'number_of_in_flight_fetch': 0,
 'task_max_waiting_in_queue_millis': 0,
 'active_shards_percent_as_number': 100.0}

In [6]:
from haystack.document_stores import OpenSearchDocumentStore

In [7]:
from haystack.nodes import TextConverter,PreProcessor

In [8]:
all_docs = convert_files_to_docs(dir_path='output', split_paragraphs=True)
preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by="word",
    split_length=10,
    split_overlap=2,
    split_respect_sentence_boundary=True,
)
docs = preprocessor.process(all_docs)

print(f"n_files_input: {len(all_docs)}\nn_docs_output: {len(docs)}")

INFO - haystack.utils.preprocessing -  Converting output\policy.txt
100%|██████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 62.87docs/s]

n_files_input: 2
n_docs_output: 90


In [9]:
    # Connect to Elasticsearch

document_store = OpenSearchDocumentStore(host=url,port=443, username=username, password=password, index="auto-insurance")

d:\exl-chatbot\env\lib\site-packages\elasticsearch\connection\http_urllib3.py:204: UserWarning: Connecting to https://search-auto-insurance-nbdvjfxbsyudd5yhpq3e4ic6bu.ap-south-1.es.amazonaws.com:443 using SSL with verify_certs=False is insecure.
  warnings.warn(
d:\exl-chatbot\env\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search-auto-insurance-nbdvjfxbsyudd5yhpq3e4ic6bu.ap-south-1.es.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
d:\exl-chatbot\env\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search-auto-insurance-nbdvjfxbsyudd5yhpq3e4ic6bu.ap-south-1.es.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(

In [10]:
document_store.write_documents(docs)

d:\exl-chatbot\env\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search-auto-insurance-nbdvjfxbsyudd5yhpq3e4ic6bu.ap-south-1.es.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [11]:
requests.get("https://search-auto-insurance-nbdvjfxbsyudd5yhpq3e4ic6bu.ap-south-1.es.amazonaws.com/auto-insurance/_count",auth=(username, password)).json()

{'count': 90,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0}}

In [12]:
print(docs[:50])

[<Document: {'content': 'How to reach us 24/7.Call, visit or e-mail your Allstate agency,Call 1-800-ALLSTATE (1-800-255-7828),Log on to create an account at allstate.com.', 'content_type': 'text', 'score': None, 'meta': {'name': 'policy.txt', '_split_id': 0}, 'embedding': None, 'id': '6afc7765e2e0843fad349e4add10b762'}>, <Document: {'content': 'How to reach us 24/7.Call, visit or e-mail your Allstate agency,Call 1-800-ALLSTATE (1-800-255-7828),Log on to create an account at allstate.com. Call your local Allstate Agent or 1-800-ALLSTATE to find out about product availability and qualifications in your state.', 'content_type': 'text', 'score': None, 'meta': {'name': 'policy.txt', '_split_id': 1}, 'embedding': None, 'id': 'd66d0a8644da45d38e665e78904e04a8'}>, <Document: {'content': 'Call your local Allstate Agent or 1-800-ALLSTATE to find out about product availability and qualifications in your state. Total premium.', 'content_type': 'text', 'score': None, 'meta': {'name': 'policy.txt', 

In [13]:
from haystack.nodes import DensePassageRetriever

In [14]:
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
    passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
    max_seq_len_query=64,
    max_seq_len_passage=256,
    batch_size=16,
    use_gpu=True,
    embed_title=True,
    use_fast_tokenizers=True,
)

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find facebook/dpr-question_encoder-single-nq-base locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...
INFO - haystack.modeling.model.language_model -  Loaded facebook/dpr-question_encoder-single-nq-base
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  ======

In [15]:
document_store.update_embeddings(retriever)

d:\exl-chatbot\env\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search-auto-insurance-nbdvjfxbsyudd5yhpq3e4ic6bu.ap-south-1.es.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
INFO - haystack.document_stores.elasticsearch -  Updating embeddings for all 90 docs ...


Updating embeddings:   0%|          | 0/90 [00:00<?, ? Docs/s]

d:\exl-chatbot\env\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search-auto-insurance-nbdvjfxbsyudd5yhpq3e4ic6bu.ap-south-1.es.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
d:\exl-chatbot\env\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search-auto-insurance-nbdvjfxbsyudd5yhpq3e4ic6bu.ap-south-1.es.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
d:\exl-chatbot\env\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search-auto-insurance-nbdvjfxbsyudd5yhpq3e4ic6bu.ap-south-1.es.amazonaws.com'. Adding certificate verification is strongl

Create embeddings:   0%|          | 0/96 [00:00<?, ? Docs/s]

d:\exl-chatbot\env\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search-auto-insurance-nbdvjfxbsyudd5yhpq3e4ic6bu.ap-south-1.es.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
